In [1]:
import os
import numpy as np
import pandas as pd
import supervision as sv
from supervision.metrics import MeanAveragePrecision

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Configuration files
model_configs = [
    # {   # Bihar to Bihar
    #     'train': 'Train Bihar',
    #     'test': 'Test Bihar',
    #     'backbone': 'swint',
    #     'head': 'rhino',
    #     'config_file': 'configs-mine/rhino-swint-dota2config/rhino_phc_haus-4scale_swint_2xb2-36e_bihar.py',
    #     'checkpoint_folder': 'work_dirs/rhino_phc_haus-4scale_swint_2xb2-36e_bihar',
    #     'val_dir': 'data/bihar/val',
    #     'inf_dir': 'results-swint/train_bihar_test_bihar',
    #     'img_height': 640,
    #     'epoch': 50,
    # },
    {
        'train': 'Delhi NCR',
        'test': 'West Bengal',
        'backbone': 'swint',
        'head': 'rhino',
        'config_file': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/configs-mine/rhino-swint-dota2config/rhino_phc_haus-4scale_swint_2xb2-36e_delhi_ncr.py',
        'checkpoint_file': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/work_dirs/rhino_phc_haus-4scale_swint_2xb2-36e_delhi_ncr/epoch_50.pth',
        'val_dir': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/data/grid_data/wb_small_airshed',
        'inf_dir': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/results-swint/delhi_to_wb',
        'img_height': 640,
        'epoch': 50,
    },
    {
        'train': 'Gen Delhi NCR (CG)',
        'test': 'West Bengal',
        'backbone': 'swint',
        'head': 'rhino',
        'config_file': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/configs-mine/rhino-swint-dota2config/rhino_phc_haus-4scale_swint_2xb2-36e_gen_delhi_CG_bks.py',
        'checkpoint_file': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/work_dirs/rhino_phc_haus-4scale_swint_2xb2-36e_gen_delhi_CG_bks/epoch_50.pth',
        'val_dir': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/data/grid_data/wb_small_airshed',
        'inf_dir': '/home/shardul.junagade/my-work/domain-adaptation-brick-kilns/RHINO/results-swint/gen_delhi_to_wb',
        'img_height': 640,
        'epoch': 50,
    }
]

In [ ]:
def get_image_names_from_directory(directory):
    """Extracts image names (without extension) from a directory."""
    return {file_name.replace(".txt", "") for file_name in os.listdir(directory) if file_name.endswith(".txt")}

def load_detections(annotations_path, img_names, is_gt=True, confidence_threshold=0):
    """Loads detections only for images that exist in both GT and Predictions."""
    sv_data = []

    for image_id in sorted(img_names):
        file_path = os.path.join(annotations_path, f"{image_id}.txt")
        if not os.path.exists(file_path):  # Ensure file exists before processing
            continue

        xyxy_list = []
        class_ids = []
        scores = []

        with open(file_path, "r") as file:
            lines = file.readlines()

        if not lines:
            detection = sv.Detections(
                xyxy=np.empty((0, 4)),
                class_id=np.empty((0,)),
                confidence=np.empty((0,)),
                metadata={"image_id": image_id}
            )
            sv_data.append(detection)
            continue
        
        for line in lines:
            data = list(map(float, line.split()))
            class_id = int(data[0])
            polygon = np.array(data[1:9]).reshape(4, 2)  # Convert to (4,2) shape
            score = data[9] if not is_gt else 1.0  # Default confidence for GT is 1.0

            if not is_gt and score < confidence_threshold:
                continue

            # Convert quadrilateral to bounding box (min x, min y, max x, max y)
            x_min, y_min = np.min(polygon, axis=0)
            x_max, y_max = np.max(polygon, axis=0)
            bbox = [x_min, y_min, x_max, y_max]

            # Append to lists
            xyxy_list.append(bbox)
            class_ids.append(class_id)
            scores.append(score)

        # Convert lists into a Supervision Detections object
        detections = sv.Detections(
            xyxy=np.array(xyxy_list),
            class_id=np.array(class_ids),
            confidence=np.array(scores),
            metadata={"image_id": image_id}
        )

        sv_data.append(detections)

    return sv_data

def get_class_counts(detections_list, num_classes=3):
    """Counts occurrences of each class in ground truth detections."""
    class_counts = np.zeros(num_classes)
    for detections in detections_list:
        unique, counts = np.unique(detections.class_id, return_counts=True)
        for cls, count in zip(unique, counts):
            # print(cls, count)
            class_counts[cls] += count
    return class_counts


In [20]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "Target State", "Backbone", "Epochs"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "Weighted mAP@50", "mAP@50:95", "mAP@50", "mAP@75", "CA mAP@50:95", "CA mAP@50", "CA mAP@75"], index=index)

In [21]:
confidence_threshold = 0.05

In [ ]:
for model_config in model_configs:
    # Load image names from directories
    backbone = model_config['backbone']
    GT_PATH = os.path.join(model_config['val_dir'], "labels")
    # PREDICTIONS_PATH = os.path.join(model_config['inf_dir'] + f"/epoch_{model_config['epoch']}_supervision_conf_0.01_nms_0.33", "annfiles")
    # PREDICTIONS_PATH = os.path.join(model_config['inf_dir'] + f"/epoch_{model_config['epoch']}", "annfiles")
    PREDICTIONS_PATH = os.path.join(model_config['inf_dir'], "annfiles")
    gt_img_names = get_image_names_from_directory(GT_PATH)
    pred_img_names = get_image_names_from_directory(PREDICTIONS_PATH)
    img_names = gt_img_names.intersection(pred_img_names)
    base_state = model_config['train']
    target_state = model_config['test']
    epoch = model_config['epoch']

    # Load GT and Predictions
    gt_data = load_detections(GT_PATH, img_names, is_gt=True)
    pred_data = load_detections(PREDICTIONS_PATH, img_names, is_gt=False, confidence_threshold=confidence_threshold)
    # print(gt_data[0])
    # print(pred_data[0])

    # Print mAP results
    print(f"\n{model_config['train']} to {model_config['test']} (Epoch {model_config['epoch']}):")
    ## mAP calculation (non-class agnostic)
    mAP_metric = MeanAveragePrecision(class_agnostic=False)
    mAP_result = mAP_metric.update(pred_data, gt_data).compute()
    # print(mAP_result)
    matched_classes = mAP_result.matched_classes.tolist()
    print(f"    Matched classes: {matched_classes}")
    # Extract overall mAP values
    mAP_50_95 = mAP_result.map50_95  # mAP 50:95
    mAP_50 = mAP_result.map50  # mAP 50
    mAP_75 = mAP_result.map75  # mAP 75
    print(f"    mAP 50:95: {mAP_50_95}, mAP 50: {mAP_50}, mAP 75: {mAP_75}")

    # Extract class-wise mAP
    # print(mAP_result.ap_per_class)
    class_wise_mAP = mAP_result.ap_per_class[:, 0].tolist()  # mAP 50:95 per class
    num_classes = 3
    final_class_wise_mAP = [0] * num_classes
    for cls, mAP in zip(matched_classes, class_wise_mAP):
        # print(f"    cls: {cls}, mAP: {mAP}")
        final_class_wise_mAP[cls] = mAP
    print(f"    class_wise_mAP: {final_class_wise_mAP}\n")
    # Calculate weighted mAP
    class_counts = get_class_counts(gt_data, num_classes=num_classes)
    print(f"    class_counts: {class_counts}")
    weighted_mAP_50 = np.sum(np.array(final_class_wise_mAP) * class_counts) / np.sum(class_counts)
    print(f"    Weighted mAP 50: {weighted_mAP_50}\n")
    

    # Compute class-agnostic mAP
    mAP_metric_agnostic = MeanAveragePrecision(class_agnostic=True)
    mAP_result_agnostic = mAP_metric_agnostic.update(pred_data, gt_data).compute()
    # Extract class-agnostic mAP values
    mAP_50_95_agnostic = mAP_result_agnostic.map50_95  # mAP 50:95
    mAP_50_agnostic = mAP_result_agnostic.map50  # mAP 50
    mAP_75_agnostic = mAP_result_agnostic.map75  # mAP 75
    print(f"    CA mAP 50:95: {mAP_50_95_agnostic}, CA mAP 50: {mAP_50_agnostic}, CA mAP 75: {mAP_75_agnostic}")

    # Update results dataframe
    result_df.loc[(base_state, target_state, backbone, epoch), :] = [f"{x:.6f}" for x in final_class_wise_mAP + [weighted_mAP_50, mAP_50_95, mAP_50, mAP_75, mAP_50_95_agnostic, mAP_50_agnostic, mAP_75_agnostic]]

In [23]:
display(result_df)

,,,,CFCBK,FCBK,Zigzag,Weighted mAP@50,mAP@50:95,mAP@50,mAP@75,CA mAP@50:95,CA mAP@50,CA mAP@75
Base State,Target State,Backbone,Epochs,,,,,,,,,,
Delhi NCR,West Bengal,swint,50,0.000000,0.001874,0.184234,0.101021,0.030113,0.093054,0.007725,0.087242,0.288924,0.021855
Gen Delhi NCR (CG),West Bengal,swint,50,0.000000,0.000000,0.198383,0.107858,0.032651,0.099191,0.019111,0.108415,0.370274,0.037028


Append to save to csv file

In [ ]:
append_to_csv = False

if append_to_csv:
    # save the dataframe as csv. if it exists, append to it
    csv_filename = f"mAP_results_{backbone}.csv"
    if os.path.exists(csv_filename):
        result_df.to_csv(csv_filename, mode='a', header=False)
    else:
        result_df.to_csv(csv_filename)

In [8]:
mAP_df_swint = pd.read_csv("mAP_results_swint.csv", index_col=[0,1,2])
display(mAP_df_swint)

,,,Epochs,CFCBK,FCBK,Zigzag,Weighted mAP@50,mAP@50:95,mAP@50,mAP@75,CA mAP@50:95,CA mAP@50,CA mAP@75
Base State,Target State,Backbone,,,,,,,,,,,
Train Bihar,Test Bihar,swint,50,0.016502,0.026997,0.600041,0.534023,0.074554,0.214513,0.023754,0.219375,0.632120,0.068042
Train Haryana,Test Bihar,swint,50,0.000000,0.002748,0.534120,0.472964,0.064248,0.178956,0.024980,0.204531,0.575262,0.077280
Train m0,Test m0,swint,50,0.136510,0.354492,0.625601,0.531033,0.116576,0.372201,0.043842,0.200259,0.691498,0.034669
Train SwinIR Bihar,Test SwinIR Bihar,swint,50,0.174917,0.033281,0.730566,0.651601,0.176001,0.312921,0.185605,0.447003,0.769618,0.476425
Train SwinIR Haryana,Test SwinIR Bihar,swint,50,0.000593,0.006664,0.519855,0.460762,0.070364,0.175704,0.042574,0.213640,0.540985,0.126694


In [11]:
mAP_df_resnet = pd.read_csv("mAP_results.csv", index_col=[0,1,2])
display(mAP_df_resnet)

CFCBK      FCBK    Zigzag  \
Base State           Target State      Epochs                                 
Train Bihar          Test Bihar        36      0.122112  0.012443  0.652780   
Train Haryana        Test Bihar        50      0.000000  0.005769  0.501982   
                                       45      0.000000  0.004587  0.504156   
Train m0             Test m0           50      0.217822  0.352871  0.531764   
                                       45      0.217822  0.362848  0.545078   
Train SwinIR Bihar   Test SwinIR Bihar 50      0.224422  0.070215  0.719853   
Train SwinIR Haryana Test SwinIR Bihar 50      0.000000  0.005867  0.424249   
                                       45      0.000000  0.006906  0.439278   
                                       40      0.000000  0.006895  0.422512   

                                               Weighted mAP@50  mAP@50:95  \
Base State           Target State      Epochs                               
Train Bihar          Test Bihar        36             0.580084   0.117900   
Train Haryana        Test Bihar        50             0.444844   0.064132   
                                       45             0.446643   0.064393   
Train m0             Test m0           50             0.469825   0.132053   
                                       45             0.481592   0.135026   
Train SwinIR Bihar   Test SwinIR Bihar 50             0.646481   0.189586   
Train SwinIR Haryana Test SwinIR Bihar 50             0.376064   0.061760   
                                       45             0.389475   0.062308   
                                       40             0.374636   0.061925   

                                                 mAP@50    mAP@75  \
Base State           Target State      Epochs                       
Train Bihar          Test Bihar        36      0.262445  0.067045   
Train Haryana        Test Bihar        50      0.169250  0.024013   
                                       45      0.169581  0.024352   
Train m0             Test m0           50      0.367486  0.059517   
                                       45      0.375249  0.062725   
Train SwinIR Bihar   Test SwinIR Bihar 50      0.338163  0.167029   
Train SwinIR Haryana Test SwinIR Bihar 50      0.143372  0.037850   
                                       45      0.148728  0.037219   
                                       40      0.143136  0.039602   

                                               CA mAP@50:95  CA mAP@50  \
Base State           Target State      Epochs                            
Train Bihar          Test Bihar        36          0.319133   0.687715   
Train Haryana        Test Bihar        50          0.192808   0.523670   
                                       45          0.193738   0.525877   
Train m0             Test m0           50          0.220652   0.584755   
                                       45          0.225493   0.599254   
Train SwinIR Bihar   Test SwinIR Bihar 50          0.429871   0.753187   
Train SwinIR Haryana Test SwinIR Bihar 50          0.209132   0.490104   
                                       45          0.208465   0.503000   
                                       40          0.210904   0.493800   

                                               CA mAP@75  
Base State           Target State      Epochs             
Train Bihar          Test Bihar        36       0.205714  
Train Haryana        Test Bihar        50       0.066662  
                                       45       0.068501  
Train m0             Test m0           50       0.125943  
                                       45       0.133003  
Train SwinIR Bihar   Test SwinIR Bihar 50       0.446126  
Train SwinIR Haryana Test SwinIR Bihar 50       0.130754  
                                       45       0.124848  
                                       40       0.133504

In [ ]:
# temp_class_counts = [  10.0,  120.0, 1000.0]
temp_class_counts = [ 17,  82, 224]
temp_class_wise_mAP = [0.676639862,	0.602386773,	0.809013]
temp_weighted_mAP = np.sum(np.array(temp_class_wise_mAP) * temp_class_counts) / np.sum(temp_class_counts)
print(temp_weighted_mAP)

0.7495897988854489


## Testing

In [12]:
# ANNOTATIONS_PATH = "gt/test_bihar_same_class_count_10_120_1000/labels"
# IMAGE_PATH = "gt/test_bihar_same_class_count_10_120_1000/images"
# PREDICTIONS_PATH = "results/train_bihar_test_bihar/annfiles"


In [13]:
# # Get common image names
# gt_images = get_image_names_from_directory(ANNOTATIONS_PATH)
# pred_images = get_image_names_from_directory(PREDICTIONS_PATH)
# common_images = sorted(gt_images.intersection(pred_images))

# print(f"Total common images: {len(common_images)}")

# # Load ground truth
# targets = load_detections(ANNOTATIONS_PATH, common_images, is_gt=True)
# print(f"Loaded {len(targets)} ground truth detections.")

# # Load predictions
# predictions = load_detections(PREDICTIONS_PATH, common_images, is_gt=False)
# print(f"Loaded {len(predictions)} prediction detections.")


In [14]:
# ## mAP calculation 
# mAP_metric = MeanAveragePrecision(class_agnostic=False)

# mAP_result=mAP_metric.update(predictions,targets).compute()
# class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
# print(f"mAP_result: {mAP_result}")
# matched_classes=mAP_result.matched_classes.tolist()
# print(f"Matched classes: {matched_classes}")
# num_classes=3
# final_class_wise_mAP = [0]*num_classes

# for cls, mAP in zip(matched_classes, class_wise_mAP):
#     print(f"cls: {cls}, mAP: {mAP}")
#     final_class_wise_mAP[cls] = mAP


# print(f"class_wise_mAP: {final_class_wise_mAP}")


# mAP_metric = MeanAveragePrecision(class_agnostic=True)
# mAP_result = mAP_metric.update(predictions, targets).compute()
# class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
# # print(f"mAP_result: {mAP_result}")
# print(f"class_wise_mAP: {class_wise_mAP}")
# print(f"class_agnostic_result: {class_agnostic_result}")

In [15]:
# base_state = "bihar"
# target_state = "bihar"
# epoch = 36

In [16]:
# index = pd.MultiIndex.from_tuples([], names=["Base State", "Target State",  "Epochs"])
# result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], index=index)
# result_df.loc[(base_state, target_state, epoch), :] = final_class_wise_mAP + [mAP_result.mAP_scores[0]] + class_agnostic_result
# display(result_df)